# Automated ML

In [29]:
import pickle
import requests
import json
import numpy as np

from azureml.core import Workspace, Experiment
from azureml.core.dataset import Dataset

from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

from azureml.core.webservice import AciWebservice, Webservice

# Dataset 
The dataset in use for this project is the Kaggle credit card fraud dataset. (https://www.kaggle.com/mlg-ulb/creditcardfraud)

This dataset includes features that are the principal coordinates that result from applying PCA to the underlying dataset. PCA was carried out to remove any sensitive data. In this experiment, I am attempting to identify fraudulent credit card transactions from the given data. This is a particularly interesting challenge, as instances of fraud make up only 0.17% of all transactions in the dataset, meaning classes are highly imbalanced.

In [2]:
ws = Workspace.from_config()

experiment_name = 'CapstoneAutoML'
project_folder = './capstone-project'

experiment=Experiment(ws, experiment_name)


# Retrieve data from datasets
key = "cc-fraud"
description_text = "Credit card fraud dataset from Kaggle."

try:
    dataset = ws.datasets[key]
except:
    print("Dataset not found in Azure datasets.") 


df = dataset.to_pandas_dataframe()
df.describe()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code AQPJSXHGA to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284806.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.841366,3.918649e-15,5.682686e-16,-8.761736e-15,2.811118e-15,-1.552103e-15,2.040130e-15,-1.698953e-15,-1.958151e-16,-3.147640e-15,...,1.471982e-16,8.042109e-16,5.282450e-16,4.458267e-15,1.426896e-15,1.701640e-15,-3.671606e-16,-1.218152e-16,88.349619,0.001727
std,47488.228330,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.250000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84691.500000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.750000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


In [3]:
amlcompute_cluster_name = "capstone"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

Creating
Succeeded.................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [4]:

automl_settings = {
    "experiment_timeout_minutes": 60,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}


automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="Class",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [5]:
remote_run = experiment.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [6]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [8]:
best_run = remote_run.get_best_child()
best_run.get_details()

{'runId': 'AutoML_d480bbf8-ce6f-49dc-92bc-a5d612a26048_38',
 'target': 'capstone',
 'status': 'Completed',
 'startTimeUtc': '2021-02-12T18:11:42.599559Z',
 'endTimeUtc': '2021-02-12T18:13:06.152733Z',
 'properties': {'runTemplate': 'automl_child',
  'pipeline_id': '__AutoML_Ensemble__',
  'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'AUC_weighted\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'CapstoneAutoML\',\'compute_target\':\'capstone\',\'subscription_id\':\'976ee174-3882-4721-b90a-b5fef6b72f24\',\'region\':\'southcentralus\',\'spark_service\':None}","ensemble_run_id":"AutoML_d480bbf8-ce6f-49dc-92bc-a5d612a26048_38","experiment_name":"CapstoneAutoML","workspace_name":"quick-starts-ws-138716","subscription_id":"976ee174-3882-4721-b90a-b5f

In [9]:
model = remote_run.register_model(model_name='capstone')

# Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.


In [38]:
   from azureml.core.model import Model
   service_name = 'cc-fraud-det-service'

   inference_config = InferenceConfig(entry_script='score.py')
   aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

   service = Model.deploy(workspace=ws,
                          name=service_name,
                          models=[model],
                          inference_config=inference_config,
                          deployment_config=aci_config,
                          overwrite=True)
   service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running..

In [37]:
# URL for the web service, should be similar to:
scoring_uri = 'http://13a915bd-cf63-4d10-895d-c637445ccd09.southcentralus.azurecontainer.io/score'

# Two sets of data to score, so we get two results back
data = {"data":
        [ 
          {
            "Time":25,
            "V1":0.1,
            "V2":-0.1,
            "V3":0.04,
            "V4":0.013,
            "V5":-0.05,
            "V6":-0.002,
            "V7":0.2,
            "V8":-0.004,
            "V9":0.012,
            "V10":0.1,
            "V11":-0.045,
            "V12":-0.2,
            "V13":0.006,
            "V14":0.09,
            "V15":-0.087,
            "V16":-0.0065,
            "V17":0.0055,
            "V18":0.02,
            "V19":-0.01,
            "V20":-0.34,
            "V21":0.003,
            "V22":0.0082,
            "V23":-0.091,
            "V24":0.0074,
            "V25":-0.0021,
            "V26":0.004,
            "V27":-0.0077,
            "V28":-0.000002,
            "Amount":25.23
          },
          {
           "Time":2,
            "V1":0.1,
            "V2":-0.1,
            "V3":0.04,
            "V4":0.013,
            "V5":-0.05,
            "V6":-0.002,
            "V7":0.2,
            "V8":-0.004,
            "V9":0.012,
            "V10":0.1,
            "V11":-0.045,
            "V12":-0.2,
            "V13":0.006,
            "V14":0.09,
            "V15":-0.087,
            "V16":-0.0065,
            "V17":0.0055,
            "V18":0.02,
            "V19":-0.01,
            "V20":-0.34,
            "V21":0.003,
            "V22":0.0082,
            "V23":-0.091,
            "V24":0.0074,
            "V25":-0.0021,
            "V26":0.004,
            "V27":-0.0077,
            "V28":-0.000002,
            "Amount":0.23 
          },
        ]   
      }
# Convert to JSON string
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)

# Set the content type
headers = {'Content-Type': 'application/json'}

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(json.loads(input_data)['data'])

{'data': [{'Time': 25, 'V1': 0.1, 'V2': -0.1, 'V3': 0.04, 'V4': 0.013, 'V5': -0.05, 'V6': -0.002, 'V7': 0.2, 'V8': -0.004, 'V9': 0.012, 'V10': 0.1, 'V11': -0.045, 'V12': -0.2, 'V13': 0.006, 'V14': 0.09, 'V15': -0.087, 'V16': -0.0065, 'V17': 0.0055, 'V18': 0.02, 'V19': -0.01, 'V20': -0.34, 'V21': 0.003, 'V22': 0.0082, 'V23': -0.091, 'V24': 0.0074, 'V25': -0.0021, 'V26': 0.004, 'V27': -0.0077, 'V28': -2e-06, 'Amount': 25.23}, {'Time': 2, 'V1': 0.1, 'V2': -0.1, 'V3': 0.04, 'V4': 0.013, 'V5': -0.05, 'V6': -0.002, 'V7': 0.2, 'V8': -0.004, 'V9': 0.012, 'V10': 0.1, 'V11': -0.045, 'V12': -0.2, 'V13': 0.006, 'V14': 0.09, 'V15': -0.087, 'V16': -0.0065, 'V17': 0.0055, 'V18': 0.02, 'V19': -0.01, 'V20': -0.34, 'V21': 0.003, 'V22': 0.0082, 'V23': -0.091, 'V24': 0.0074, 'V25': -0.0021, 'V26': 0.004, 'V27': -0.0077, 'V28': -2e-06, 'Amount': 0.23}]}


In [24]:
name = "cc-fraud-det-service"

# load existing web service
service = Webservice(name=name, workspace=ws)
logs = service.get_logs()

for line in logs.split('\n'):
    print(line)

2021-02-12T18:25:35,967968420+00:00 - rsyslog/run 
2021-02-12T18:25:35,968382045+00:00 - iot-server/run 
2021-02-12T18:25:35,968943679+00:00 - gunicorn/run 
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2021-02-12T18:25:35,970955802+00:00 - nginx/run 
rsyslogd